In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
#import xgboost as xgb
import lightgbm as lgb


In [2]:
from imp import reload
import matplotlib.pyplot as plt  
reload(plt)

<module 'matplotlib.pyplot' from 'D:\\ProgramData\\anaconda\\lib\\site-packages\\matplotlib\\pyplot.py'>

### 读取phishing data

In [3]:
phishing_data = pd.read_csv('./data/all_phishing_data.csv')

In [4]:
len(phishing_data)

70898

In [5]:
#phishing_data.groupby(['to'])['from'].agg(['count', 'nunique'])

In [6]:
#train_data.groupby(['to'])['from'].agg(['count', 'nunique']).mean()

In [7]:
#还未划分train test
frac = 0.001
def feature_matrix_to_train_matrix(features, train_label):
    train = pd.merge(left=features, right=train_label, left_on='address', right_on='address', how='inner')
    black_sample = train[train['label'] == 1]
    white_sample = train[train['label'] == 0].sample(frac = frac)
    train = pd.concat([black_sample, white_sample])
    
    #打乱黑白样本
    train = train.sample(frac=1)
    #data = pd.merge(left=features, right=train_label, left_on='from', right_on='address', how='inner')
    #train = data[:int(len(data) * frac)]
    #test = data[int(len(data) * frac):]
    X_train = train.drop(['address', 'label'], axis=1)
    #X_train = X_train.sample(frac=frac, axis=1)
    y_train = train['label']
    #X_test = test.drop(['from', 'to', 'address', 'label'], axis=1)
    #y_test = test_label['label']
    return X_train, y_train

In [8]:
#y_train.sum() / len(y_train)

In [9]:
def wei2ether(x):
    x = list('0'.zfill(18) + x)
    x.insert(-18, '.')
    return np.double(''.join(x))

In [10]:
train_data = pd.read_csv('./data/train_data_ver2.0.csv')
train_label = pd.read_csv('./data/train_label_ver2.0.csv')

In [11]:
train_data['value'] = train_data['value'].apply(wei2ether)

In [12]:
train_label['label'].value_counts()

0    9099062
1       1220
Name: label, dtype: int64

In [13]:
len(train_data)

30956458

## 观察数据

In [14]:
len(set(train_data['to']))

8639415

In [15]:
def get_trival_feature(train_data):
    #feature between from and block:
    from_block_feature = train_data.groupby(['from'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('from_block_')
    from_value_feature = train_data.groupby(['from'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('from_value_')
    from_feature = pd.concat([from_block_feature, from_value_feature], axis = 1)
    #from_feature = from_feature.merge(left_on='from', right=train_address, right_on='address', how='right')
    to_block_feature = train_data.groupby(['to'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('to_block_')
    to_value_feature = train_data.groupby(['to'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('to_value_')
    to_feature = pd.concat([to_block_feature, to_value_feature], axis = 1)
    from_feature.reset_index(inplace=True)
    to_feature.reset_index(inplace=True)
    #to_feature = to_feature.merge(left_on='to', right=train_address, right_on='address', how='right')
    trival_feature = pd.merge(left=from_feature, right=to_feature, left_on='from', right_on='to', how='inner')
    trival_feature['address'] = trival_feature['from']
    trival_feature.drop(['from', 'to'], axis = 1, inplace=True)
    return trival_feature

In [16]:
train_address =train_label['address']

In [17]:
#pre sampled
train_data

,blockNumber,from,to,value
0,1917029,0xcf5cdd85813129271c907348325c013847674fc6,0x60aa80fe41b43c180870a137c7a87334e5c3cb5d,2.100000e-01
1,1917031,0x63656b599cb67a18b5be74f7dcd269137a6efdcb,0x7180eb39a6264938fdb3effd7341c4727c382153,9.995800e-01
2,1917031,0x63656b599cb67a18b5be74f7dcd269137a6efdcb,0x7180eb39a6264938fdb3effd7341c4727c382153,9.995800e-01
3,1917032,0xea321e7e27877e4e623a74b1262f184f404a8319,0x1e639f11abdd00fca7779aab75c3c3ed10c0d1a0,8.900000e-01
4,1917032,0x60aa80fe41b43c180870a137c7a87334e5c3cb5d,0xcf5cdd85813129271c907348325c013847674fc6,1.000000e-18
...,...,...,...,...
30956453,7006697,0xcdfaa6fd2461d7129a7fdc193c9d97504ff3b6f6,0x3b00a158410514fc4bbd40bf51f6fd0d140fb45d,1.800000e-01
30956454,7006697,0x20e6f8babbfdf132d049814698ec2281ae4b3da3,0xebe68414ab9a7d9124b685c9bb6f70d553a01664,1.789200e-03
30956455,7006699,0x29bc381c5a88d0fea8f60818725acceee9b981e7,0x968c6686b9d4f8f2aed5a6c5ea629f67cfc9ac2e,1.400000e+01
30956456,7006699,0xa2d3d545de4235044fff5116a4f8a043aaf13821,0x25bddd8c28d657f190d8e32cda1e29207718a7da,2.910000e+00


In [18]:
#feature between from and block:
from_block_feature = train_data.groupby(['from'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('from_block_')

In [19]:
from_value_feature = train_data.groupby(['from'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('from_value_')

In [20]:
from_feature = pd.concat([from_block_feature, from_value_feature], axis = 1)
#from_feature = from_feature.merge(left_on='from', right=train_address, right_on='address', how='right')
#from_feature.fillna(0, inplace=True)

In [21]:
to_block_feature = train_data.groupby(['to'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('to_block_')
to_value_feature = train_data.groupby(['to'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('to_value_')

In [22]:
to_feature = pd.concat([to_block_feature, to_value_feature], axis = 1)
#to_feature = to_feature.merge(left_on='to', right=train_address, right_on='address', how='right')

In [23]:
from_feature.reset_index(inplace=True)
to_feature.reset_index(inplace=True)

In [24]:
#merge方式

trival_feature = pd.merge(left=from_feature, right=to_feature, left_on='from', right_on='to', how='inner')

In [25]:
trival_feature

,from,from_block_ptp,from_block_std,from_value_sum,from_value_mean,from_value_std,from_value_max,from_value_min,to,to_block_ptp,to_block_std,to_value_sum,to_value_mean,to_value_std,to_value_max,to_value_min
0,0x0000000048246c5f37df029bef0cefd9ee36f301,35020,16237.896054,1.906650e-01,4.766626e-02,0.055247,1.282759e-01,9.346600e-03,0x0000000048246c5f37df029bef0cefd9ee36f301,93379,38911.802283,0.079356,0.019839,0.008781,0.029139,0.010000
1,0x000000004fdd41501a42651743c55f7e41ec85e4,322138,144046.283394,4.281103e-02,1.070276e-02,0.012553,2.620300e-02,1.000000e-05,0x000000004fdd41501a42651743c55f7e41ec85e4,0,NaN,0.054110,0.054110,NaN,0.054110,0.054110
2,0x00000000666bc8d3212838a58e92c18e6cfe9ef1,0,NaN,8.888889e-11,8.888889e-11,NaN,8.888889e-11,8.888889e-11,0x00000000666bc8d3212838a58e92c18e6cfe9ef1,1168154,826009.614870,0.070910,0.035455,0.050127,0.070900,0.000010
3,0x00000000ad4eb06bc550dc45ce28888c0626839a,80,22.360860,7.324813e+01,1.331784e+00,1.810276,9.183630e+00,1.006096e-01,0x00000000ad4eb06bc550dc45ce28888c0626839a,0,NaN,103.500000,103.500000,NaN,103.500000,103.500000
4,0x00000000b9d54fb8ab1a28ebd2ccbfe3247e517f,0,NaN,2.669227e+00,2.669227e+00,NaN,2.669227e+00,2.669227e+00,0x00000000b9d54fb8ab1a28ebd2ccbfe3247e517f,270001,145332.203372,43.790000,14.596667,6.829863,19.900000,6.890000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7587228,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,2636291,418260.353952,5.332489e+03,2.866930e+01,144.015450,1.597000e+03,1.000000e-02,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,2055331,525883.200147,3136.556590,13.696754,35.252536,380.000000,0.001000
7587229,0xfffffe16439027b253808b7a55c787b67c22a75d,15,8.386497,1.009981e+02,3.366604e+01,48.429568,8.922260e+01,3.629000e-01,0xfffffe16439027b253808b7a55c787b67c22a75d,0,0.000000,101.000000,50.500000,54.158249,88.795665,12.204335
7587230,0xffffffa5473301b0489d7e1d1e7786b4b79e2cc0,117025,40840.724921,1.328789e+00,1.476433e-01,0.211307,5.500000e-01,3.000000e-03,0xffffffa5473301b0489d7e1d1e7786b4b79e2cc0,110058,39841.898372,0.165020,0.023574,0.015793,0.050000,0.004469
7587231,0xfffffff1ac8dcf49f4afd3f90995120e568740f9,696510,189335.232954,4.959864e+02,3.815280e+01,90.528909,2.500000e+02,9.864391e-01,0xfffffff1ac8dcf49f4afd3f90995120e568740f9,527774,28389.016979,1524.310398,3.928635,13.851119,200.000000,0.001000


In [26]:
trival_feature['address'] = trival_feature['from']
trival_feature.drop(['from', 'to'], axis = 1, inplace=True)

## 构建特征

### 网络特征——from, to

In [27]:
def get_network_feature(train_data):
    from_to_one_degree_feature = train_data.groupby(['from'])['to'].agg(['count', 'nunique']).add_prefix('from_to_').reset_index()
    from_to_one_degree_feature['from_to_count_nunique_ratio'] = from_to_one_degree_feature['from_to_count'] / from_to_one_degree_feature['from_to_nunique']
    from_to_one_degree_feature['from_to_count_nunique_equal'] = (from_to_one_degree_feature['from_to_count'] == from_to_one_degree_feature['from_to_nunique']).astype(int)
    from_to_two_degree_feature = train_data.groupby(['from', 'to'])['value'].agg(['sum', 'mean', 'max', 'min']).groupby(['from']).agg(
        ['sum', 'std', 'median', 'max', 'min']).add_prefix('from_to_')
    from_to_two_degree_feature = pd.DataFrame(from_to_two_degree_feature.to_records(), index=None)
    from_to_degree_feature =  from_to_two_degree_feature.merge(right=from_to_one_degree_feature, left_on='from', right_on='from', how='inner')
                                                
        #to,from 特征以相同方式提取
    to_from_one_degree_feature = train_data.groupby(['to'])['from'].agg(['count', 'nunique']).add_prefix('to_from_').reset_index()
    to_from_one_degree_feature['to_from_count_nunique_ratio'] = to_from_one_degree_feature['to_from_count'] / to_from_one_degree_feature['to_from_nunique']
    to_from_one_degree_feature['to_from_count_nunique_equal'] = (to_from_one_degree_feature['to_from_count'] == to_from_one_degree_feature['to_from_nunique']).astype(int)
    
    to_from_two_degree_feature = train_data.groupby(['to', 'from'])['value'].agg(['sum', 'mean', 'max', 'min']).groupby(['to']).agg(
    ['sum', 'std', 'median', 'max', 'min']).add_prefix('to_from_')
    to_from_two_degree_feature = pd.DataFrame(to_from_two_degree_feature.to_records(), index=None)
    to_from_degree_feature =  to_from_two_degree_feature.merge(right=to_from_one_degree_feature, left_on='to', right_on='to', how='inner')
    return from_to_degree_feature, to_from_degree_feature

In [28]:
#base: 每个from 节点对应的 to 节点的（不同）个数 (one_degree)
from_to_one_degree_feature = train_data.groupby(['from'])['to'].agg(['count', 'nunique']).add_prefix('from_to_').reset_index()
from_to_one_degree_feature['from_to_count_nunique_ratio'] = from_to_one_degree_feature['from_to_count'] / from_to_one_degree_feature['from_to_nunique']
from_to_one_degree_feature['from_to_count_nunique_equal'] = (from_to_one_degree_feature['from_to_count'] == from_to_one_degree_feature['from_to_nunique']).astype(int)
# 还可做： count/ nunique, count == nunique?

In [29]:
#统计from 节点相连接的 to 节点的信息情况

In [30]:
from_to_two_degree_feature = train_data.groupby(['from', 'to'])['value'].agg(['sum', 'mean', 'max', 'min']).groupby(['from']).agg(
    ['sum', 'std', 'median', 'max', 'min']).add_prefix('from_to_')
from_to_two_degree_feature = pd.DataFrame(from_to_two_degree_feature.to_records(), index=None)

In [31]:
from_to_degree_feature =  from_to_two_degree_feature.merge(right=from_to_one_degree_feature, left_on='from', right_on='from', how='inner')

In [32]:
#to,from 特征以相同方式提取
to_from_one_degree_feature = train_data.groupby(['to'])['from'].agg(['count', 'nunique']).add_prefix('to_from_').reset_index()
to_from_one_degree_feature['to_from_count_nunique_ratio'] = to_from_one_degree_feature['to_from_count'] / to_from_one_degree_feature['to_from_nunique']
to_from_one_degree_feature['to_from_count_nunique_equal'] = (to_from_one_degree_feature['to_from_count'] == to_from_one_degree_feature['to_from_nunique']).astype(int)

In [33]:
to_from_two_degree_feature = train_data.groupby(['to', 'from'])['value'].agg(['sum', 'mean', 'max', 'min']).groupby(['to']).agg(
    ['sum', 'std', 'median', 'max', 'min']).add_prefix('to_from_')
to_from_two_degree_feature = pd.DataFrame(to_from_two_degree_feature.to_records(), index=None)

In [34]:
to_from_degree_feature =  to_from_two_degree_feature.merge(right=to_from_one_degree_feature, left_on='to', right_on='to', how='inner')

### 特征拼接

In [35]:
def get_feature(train_data):
    trival_feature = get_trival_feature(train_data)
    from_to_degree_feature, to_from_degree_feature = get_network_feature(train_data)
    feature = trival_feature.merge(right=from_to_degree_feature, left_on='address', right_on='from', how='left')
    feature = feature.merge(right=to_from_degree_feature, left_on='address', right_on='to',how='left')
    feature.drop(['from','to'], inplace=True, axis = 1)
    return feature

In [36]:
feature = trival_feature.merge(right=from_to_degree_feature, left_on='address', right_on='from', how='left')

In [37]:
feature = feature.merge(right=to_from_degree_feature, left_on='address', right_on='to',how='left')

In [38]:
feature.drop(['from','to'], inplace=True, axis = 1)

In [39]:
def deal_feature_name(x):
    if x.count('from_to_') == 2:
        return 'from_in_' + x[1:-1].replace('from_to_', '').replace(',', '').replace('\'', '').replace(' ', '_')
    if x.count('to_from_') == 2:
        return 'to_out_' + x[1:-1].replace('to_from_', '').replace(',', '').replace('\'', '').replace(' ', '_')
    if x.count('from_to_') == 1:
        return x.replace('from_to', 'out')
    if x.count('to_from_') == 1:
        return x.replace('to_from', 'in')
    if (x.count('block') == 1 or x.count('value') == 1):
        return x.replace('from_block', 'out_block').replace('from_value', 'out_value').replace('to_block', 'in_block').replace('to_value', 'in_value')
    return x

In [40]:
feature = get_feature(train_data)
#处理特征名称
feature.columns = list(map(deal_feature_name,feature.columns))

In [41]:
feature.columns

Index(['out_block_ptp', 'out_block_std', 'out_value_sum', 'out_value_mean',
       'out_value_std', 'out_value_max', 'out_value_min', 'in_block_ptp',
       'in_block_std', 'in_value_sum', 'in_value_mean', 'in_value_std',
       'in_value_max', 'in_value_min', 'address', 'from_in_sum_sum',
       'from_in_sum_std', 'from_in_sum_median', 'from_in_sum_max',
       'from_in_sum_min', 'from_in_mean_sum', 'from_in_mean_std',
       'from_in_mean_median', 'from_in_mean_max', 'from_in_mean_min',
       'from_in_max_sum', 'from_in_max_std', 'from_in_max_median',
       'from_in_max_max', 'from_in_max_min', 'from_in_min_sum',
       'from_in_min_std', 'from_in_min_median', 'from_in_min_max',
       'from_in_min_min', 'out_count', 'out_nunique',
       'out_count_nunique_ratio', 'out_count_nunique_equal', 'to_out_sum_sum',
       'to_out_sum_std', 'to_out_sum_median', 'to_out_sum_max',
       'to_out_sum_min', 'to_out_mean_sum', 'to_out_mean_std',
       'to_out_mean_median', 'to_out_mean_max', 

In [42]:
X_train, y_train = feature_matrix_to_train_matrix(feature, train_label)

In [43]:
len(y_train)

8385

## 模型测试

In [44]:
threshold = 0.5
def lgb_evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    pre.preds=pre.preds.map(lambda x: 1 if x>=threshold else 0)
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    precision = metrics.precision_score(pre.label,pre.preds)
    recall = metrics.recall_score(pre.label,pre.preds)
    res = auc
    return 'res',res, True

def lgb_pre_evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    pre.preds=pre.preds.map(lambda x: 1 if x>=threshold else 0)
    precision = metrics.precision_score(pre.label,pre.preds)
    res = precision
    return 'res',res, True

def lgb_recall_evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    pre.preds=pre.preds.map(lambda x: 1 if x>=threshold else 0)
    recall = metrics.recall_score(pre.label,pre.preds)
    res = recall
    return 'res',res, True

def lgb_auc_evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    pre.preds=pre.preds.map(lambda x: 1 if x>=threshold else 0)
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    res = auc
    return 'res',res, True

In [45]:
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    #'colsample_bytree': 0.8,
    'learning_rate': 0.2,
    'num_leaves': 8,
    'verbosity': 1
    }

In [46]:
dtrain = lgb.Dataset(X_train, label = y_train)

In [47]:
X_train

,out_block_ptp,out_block_std,out_value_sum,out_value_mean,out_value_std,out_value_max,out_value_min,in_block_ptp,in_block_std,in_value_sum,...,to_out_max_min,to_out_min_sum,to_out_min_std,to_out_min_median,to_out_min_max,to_out_min_min,in_count,in_nunique,in_count_nunique_ratio,in_count_nunique_equal
3170878,0,NaN,24.044183,24.044183,NaN,24.044183,24.044183,2,1.414214,101.00,...,29.486299,101.00,29.717861,50.50,71.513701,29.486299,2,2,1.0,1
4669941,11,7.778175,555.906982,277.953491,381.213026,547.511807,8.395175,1,0.707107,2001.00,...,369.402991,2001.00,892.505949,1000.50,1631.597009,369.402991,2,2,1.0,1
2318236,10,4.509250,100.998320,25.249580,48.855623,98.513346,0.000960,2,1.414214,101.00,...,19.083959,101.00,44.428991,50.50,81.916041,19.083959,2,2,1.0,1
370213,527611,267325.215747,5.635785,1.878595,1.848813,3.750000,0.053233,17787,9103.289094,4.04,...,1.850000,1.09,NaN,1.09,1.090000,1.090000,3,1,3.0,0
4814803,1,0.707107,90.998740,45.499370,4.336853,48.565988,42.432752,0,0.000000,101.00,...,23.092228,101.00,38.760442,50.50,77.907772,23.092228,2,2,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7546493,27,12.449900,1000.998320,250.249580,308.442450,635.067477,0.000960,0,0.000000,1001.00,...,265.595539,1001.00,332.205075,500.50,735.404461,265.595539,2,2,1.0,1
5677870,0,0.000000,1000.484718,500.242359,230.434229,663.183965,337.300753,1,0.707107,1001.00,...,87.018153,1001.00,584.751636,500.50,913.981847,87.018153,2,2,1.0,1
5387707,0,0.000000,1000.512713,500.256357,526.009404,872.201173,128.311540,0,0.000000,1001.00,...,66.453688,1001.00,613.834180,500.50,934.546312,66.453688,2,2,1.0,1
2474913,4,2.828427,97.392500,48.696250,68.865540,97.391540,0.000960,2,1.414214,101.00,...,18.576265,101.00,45.146979,50.50,82.423735,18.576265,2,2,1.0,1


In [53]:
cv_res = lgb.cv(lgb_params, dtrain, num_boost_round=250,feval=lgb_evalMetric, verbose_eval = 10, nfold=3,metrics=('lgb_evalMetric'),
       early_stopping_rounds= 100)

[LightGBM] [Info] Number of positive: 533, number of negative: 5057
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004026 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14267
[LightGBM] [Info] Number of data points in the train set: 5590, number of used features: 62
[LightGBM] [Info] Number of positive: 533, number of negative: 5057
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004058 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14267
[LightGBM] [Info] Number of data points in the train set: 5590, number of used features: 62
[LightGBM] [Info] Number of positive: 532, number of negative: 5058
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004196 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14267
[LightGBM] [

D:\ProgramData\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ProgramData\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is i

[10]	cv_agg's res: 0.827136 + 0.00713056
[20]	cv_agg's res: 0.845455 + 0.00545512
[30]	cv_agg's res: 0.851153 + 0.00619371
[40]	cv_agg's res: 0.852767 + 0.0064296
[50]	cv_agg's res: 0.858623 + 0.00687808
[60]	cv_agg's res: 0.860173 + 0.00636969
[70]	cv_agg's res: 0.860171 + 0.00742489
[80]	cv_agg's res: 0.859909 + 0.0059465
[90]	cv_agg's res: 0.859622 + 0.00111831
[100]	cv_agg's res: 0.858239 + 0.00181924
[110]	cv_agg's res: 0.859285 + 0.0064814
[120]	cv_agg's res: 0.859285 + 0.00781355
[130]	cv_agg's res: 0.858331 + 0.00494462
[140]	cv_agg's res: 0.862354 + 0.00615874
[150]	cv_agg's res: 0.862976 + 0.00751849
[160]	cv_agg's res: 0.864493 + 0.0059436
[170]	cv_agg's res: 0.864926 + 0.00635445
[180]	cv_agg's res: 0.865687 + 0.00685875
[190]	cv_agg's res: 0.864239 + 0.00917733
[200]	cv_agg's res: 0.861742 + 0.00942812
[210]	cv_agg's res: 0.859106 + 0.00643618
[220]	cv_agg's res: 0.85993 + 0.00829571
[230]	cv_agg's res: 0.862168 + 0.00668023
[240]	cv_agg's res: 0.860423 + 0.00870115
[250]	

In [49]:
max(cv_res['res-mean'])

0.8599614243722938

In [ ]:
while True:    
    X_train, y_train = feature_matrix_to_train_matrix(feature, train_label)
    dtrain = lgb.Dataset(X_train, label = y_train)
    evals_result = {}
    lgb_model = lgb.train(lgb_params, dtrain, num_boost_round=400,feval=lgb_evalMetric, verbose_eval = 10,valid_sets=dtrain,evals_result=evals_result,#非常重要的参数,一定要明确设置
           early_stopping_rounds= 100)
    sorted_impo = pd.DataFrame([lgb_model.feature_name(), lgb_model.feature_importance()]).T.sort_values(by=1, ascending=False)[0]
    if (sorted_impo[1] == 'to_out_sum_median' and sorted_impo[2] == 'from_in_sum_mean') :break

In [ ]:
lgb.plot_importance(lgb_model, max_num_features=15, title='')
plt.savefig('importance.jpg')

In [ ]:
sorted_impo = pd.DataFrame([lgb_model.feature_name(), lgb_model.feature_importance()]).T.sort_values(by=1, ascending=False)[0]

### 动态性测试

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#按時間劃分
cv_res = []
frac_num = 15
for i in range(frac_num):
    curr_train = train_data[0:int(len(train_data) * ((i + 1) / frac_num))]

    curr_feature = get_feature(curr_train)

    X_curr, y_curr = feature_matrix_to_train_matrix(curr_feature, train_label)

    dtrain = lgb.Dataset(X_curr, label = y_curr)

    cv_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_evalMetric'),
           early_stopping_rounds= 100)

    cv_res.append(max(cv_tmp['res-mean']))

In [ ]:
cv_res

In [ ]:
phishing_label = pd.read_csv('./data/phishing_label.csv')

In [ ]:
len(train_data)

In [ ]:
address_map = {}
def cal_data_index(x):
    if x in address_map:
        address_map[x] = address_map[x] + 1
    else:
        address_map[x] = 1
    return address_map[x]

In [ ]:
train_data['data_index'] = train_data['to'].apply(cal_data_index)

In [ ]:
upperbound = 59
cv_pre_res = []
cv_recall_res = []
cv_auc_res = []
for i in range(upperbound):
    kth_train_address_set = set(train_data[train_data['data_index'] == i + 1]['to'])
    kth_train_data = train_data[train_data['data_index'] <= i + 1]
    kth_train_data = kth_train_data[kth_train_data['to'].isin(kth_train_address_set)]
    kth_train_feature = get_feature(kth_train_data)
    X_kth, y_kth = feature_matrix_to_train_matrix(kth_train_feature, train_label)
    dtrain = lgb.Dataset(X_kth, label = y_kth)
    cv_pre_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_pre_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_pre_evalMetric'),
           early_stopping_rounds= 100)
    cv_recall_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_recall_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_recall_evalMetric'),
           early_stopping_rounds= 100)   
    cv_auc_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_auc_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_auc_evalMetric'),
           early_stopping_rounds= 100)      
    cv_pre_res.append(max(cv_pre_tmp['res-mean']))
    cv_recall_res.append(max(cv_recall_tmp['res-mean']))
    cv_auc_res.append(max(cv_auc_tmp['res-mean']))

In [ ]:
cv_f1_res = list(map(lambda x, y:  2 * x * y / (x + y), cv_pre_res, cv_recall_res))

In [ ]:
cv_pre_res[15]

In [ ]:
index = list(range(len(cv_auc_res)))

In [ ]:
import matplotlib.pyplot as plt
x = np.array(index)


y_pre = np.array(cv_pre_res)
f_fit = np.poly1d(np.polyfit(x, y_pre, 12))
yfit = f_fit(x)
plt.plot(x, yfit, label='Precision')

y_recall = np.array(cv_recall_res)
f_fit = np.poly1d(np.polyfit(x, y_recall, 7))
yfit = f_fit(x)
plt.plot(x, yfit, label='Recall')

y_f1 = np.array(cv_f1_res)
f_fit = np.poly1d(np.polyfit(x, y_f1, 10))
yfit = f_fit(x)
plt.plot(x, yfit, label='F1')

y_auc = np.array(cv_auc_res)
f_fit = np.poly1d(np.polyfit(x, y_auc, 10))
yfit = f_fit(x)
plt.plot(x, yfit, label='AUC')

plt.xlabel('Number of in transaction')
plt.ylabel('Cross validation performance')
plt.legend()
plt.savefig('dynamic_performance.jpg')

In [ ]:
train_label.sum()

In [ ]:
feature_new = get_feature(train_data)

In [ ]:
X_tmp, y_tmp = feature_matrix_to_train_matrix(feature, train_label)
dtrain = lgb.Dataset(X_tmp, label = y_tmp)
cv_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_evalMetric, verbose_eval = 10, nfold=5,metrics=('lgb_evalMetric'),
           early_stopping_rounds= 100)

In [ ]:
y_tmp.sum()

### 與svm和dt對比

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_tn, x_tt, y_tn, y_tt = train_test_split(X_train,y_train, test_size=0.25)

In [ ]:
from sklearn import svm

In [ ]:
from sklearn import tree
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
x_tn.fillna(0, inplace=True)
x_tt.fillna(0, inplace=True)
clf.fit(x_tn, y_tn)
svm_preds = clf.predict(x_tt)
svm_auc_score = roc_auc_score(y_tt,svm_preds)
svm_precision_score = precision_score(y_tt, svm_preds, average='macro')
svm_recall_score = recall_score(y_tt, svm_preds, average='macro')

In [ ]:
clf = tree.DecisionTreeClassifier()
x_tn.fillna(0, inplace=True)
x_tt.fillna(0, inplace=True)
clf.fit(x_tn, y_tn)
dt_preds = clf.predict(x_tt)
dt_auc_score = roc_auc_score(y_tt,dt_preds)
dt_precision_score = precision_score(y_tt, dt_preds, average='macro')
dt_recall_score = recall_score(y_tt, dt_preds, average='macro')

In [ ]:
#经过均衡
print(dt_auc_score)
print(dt_precision_score)
print(dt_recall_score)

## 案例分析

In [ ]:
sample_addr = '0x2a6d8021861f27ab992572d8689017b7a83c989d'

In [ ]:
sample_addr_data = train_data[(train_data['from'] == sample_addr) | (train_data['to'] == sample_addr)]

In [ ]:
sample_addr_data

In [ ]:
#sample_addr_data

In [ ]:
sample_dynamic_feature = pd.DataFrame()
for i in range(len(sample_addr_data)):
    part_sample_addr_data = sample_addr_data[:i]
    tmp_fea = get_sample_feature(part_sample_addr_data)
    target_fea = tmp_fea[tmp_fea['address'] == sample_addr]
    sample_dynamic_feature = pd.concat([sample_dynamic_feature,target_fea])

In [ ]:
sample_dynamic_feature.drop('address', axis = 1, inplace=True)

In [ ]:
without_sample_fea = feature[feature['address'] != sample_addr]
X_tmp, y_tmp = feature_matrix_to_train_matrix(without_sample_fea, train_label)
dtrain = lgb.Dataset(X_tmp, label = y_tmp)
sample_lgb_model = lgb.train(lgb_params, dtrain, num_boost_round=400,feval=lgb_evalMetric, verbose_eval = 10,valid_sets=dtrain,
       early_stopping_rounds= 100)

In [ ]:
sample_dynamic_res = sample_lgb_model.predict(sample_dynamic_feature)

In [ ]:
sample_dynamic_res

In [ ]:
dynamic_x = np.array(range(len(sample_dynamic_res)))
dynamic_f_fit = np.poly1d(np.polyfit(dynamic_x, sample_dynamic_res,5))
dynamic_yfit = dynamic_f_fit(dynamic_x)
plt.plot(dynamic_x, dynamic_yfit, marker='o', markevery=[3,15])
plt.axhline(y=0.52306926,ls='--', xmax=0.06, color='r')
plt.axvline(x=3, ls='--',ymax=0.25, color='r')
plt.annotate("(4, 0.523)", xy=(4, 0.52), xytext=(4, 2), textcoords='offset points')
plt.axhline(y=0.9078484 ,ls='--', xmax=0.15,color='r')
plt.axvline(x=15, ls='--',ymax=0.77,color='r')
plt.annotate("(16, 0.908)", xy=(15, 0.908), xytext=(6, 2), textcoords='offset points')
plt.xlabel('Number of transactions')
plt.ylabel('P(phishing address)')
plt.savefig('P_phishing.jpg')

In [ ]:
dynamic_yfit

In [ ]:
def get_sample_trival_feature(train_data):
    #feature between from and block:
    from_block_feature = train_data.groupby(['from'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('from_block_')
    from_value_feature = train_data.groupby(['from'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('from_value_')
    from_feature = pd.concat([from_block_feature, from_value_feature], axis = 1)
    #from_feature = from_feature.merge(left_on='from', right=train_address, right_on='address', how='right')
    to_block_feature = train_data.groupby(['to'])['blockNumber'].agg([np.ptp, 'std']).add_prefix('to_block_')
    to_value_feature = train_data.groupby(['to'])['value'].agg(['sum', 'mean', 'std', 'max', 'min']).add_prefix('to_value_')
    to_feature = pd.concat([to_block_feature, to_value_feature], axis = 1)
    from_feature.reset_index(inplace=True)
    to_feature.reset_index(inplace=True)
    #to_feature = to_feature.merge(left_on='to', right=train_address, right_on='address', how='right')
    trival_feature = pd.merge(left=from_feature, right=to_feature, left_on='from', right_on='to', how='right')
    trival_feature['address'] = trival_feature['to']
    trival_feature.drop(['from', 'to'], axis = 1, inplace=True)
    return trival_feature
def get_sample_feature(train_data):
    trival_feature = get_sample_trival_feature(train_data)
    from_to_degree_feature, to_from_degree_feature = get_network_feature(train_data)
    feature = trival_feature.merge(right=from_to_degree_feature, left_on='address', right_on='from', how='left')
    feature = feature.merge(right=to_from_degree_feature, left_on='address', right_on='to',how='left')
    feature.drop(['from','to'], inplace=True, axis = 1)
    return feature

### phishing 金额对比

In [ ]:
sample_addr_data.iloc[0]['value']

In [ ]:
phishing_sum = [sample_addr_data.iloc[0]['value']]
model_expected_sum = [sample_addr_data.iloc[0]['value']]
for i in range(1, len(sample_addr_data) - 1):
    phishing_sum.append(phishing_sum[-1] + sample_addr_data.iloc[i]['value'])
    model_expected_sum.append(model_expected_sum[-1] + sample_addr_data.iloc[i]['value'] * (1 - sample_dynamic_res[i - 1]))

In [ ]:
pd.Series(phishing_sum).plot(label='Loss without model warning')
pd.Series(model_expected_sum).plot(label='Loss with model warning')
plt.xlabel('Number of transactions')
plt.ylabel('Loss(Eth)')
plt.legend()
plt.savefig('Loss_with_model.jpg')

In [ ]:
1 - model_expected_sum[-1] / phishing_sum[-1]

In [ ]:
tmp_pre = 0.819627554383652

tmp_recall = 0.8049612759757687
(tmp_pre * tmp_recall * 2) / (tmp_pre + tmp_recall)

In [ ]:
all_address_set = set(train_data['from']).union(set(train_data['from']))

In [ ]:
len(all_address_set)

In [ ]:
pd.DataFrame(lgb_model.feature_name()).to_csv('feature_names.csv')

## 不同feature frac 的影响

In [ ]:

feature_frac_precision = []
feature_frac_recall = []
feature_frac_auc = []
for i in range(20):
    X_train, y_train = feature_matrix_to_train_matrix(feature, train_label)
    dtrain = lgb.Dataset(X_train, label = y_train)
    feature_frac_precision_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_pre_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_pre_evalMetric'),
           early_stopping_rounds= 100)
    feature_frac_recall_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_recall_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_recall_evalMetric'),
           early_stopping_rounds= 100)   
    feature_frac_auc_tmp = lgb.cv(lgb_params, dtrain, num_boost_round=400,feval=lgb_auc_evalMetric, verbose_eval = 10, nfold=2,metrics=('lgb_auc_evalMetric'),
           early_stopping_rounds= 100)      
    feature_frac_precision.append(max(feature_frac_precision_tmp['res-mean']))
    feature_frac_recall.append(max(feature_frac_recall_tmp['res-mean']))
    feature_frac_auc.append(max(feature_frac_auc_tmp['res-mean']))

In [ ]:
np.mean(feature_frac_precision)

In [ ]:
np.mean(feature_frac_recall)

In [ ]:
np.mean(feature_frac_auc)